In [18]:
import pandas as pd
import openai
import time
import concurrent.futures
from concurrent.futures import ThreadPoolExecutor
from openai.error import OpenAIError

In [19]:
# Configure OpenAI API credentials
openai.api_key = "xxxxxxxxxxxxxxxxxx"

In [24]:
def process_review(review):
    messages = [
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": f"A narrative is a piece of text with a clear beginning, middle, and end. It typically involves a conflict that arises and is later resolved. Please give a rating only on how much the following review resembles a narrative, on a scale of 1 to 10. No explanations.Give narrativity score in the format Narrativity score:"},
        {"role": "assistant", "content": review[1]}
    ]
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=messages,
        temperature=0,
        n=1
    )
    assistant_reply = response.choices[0].message["content"]
    rating = assistant_reply.strip()
    return [review[0], review[1], rating]

def func(narrativity):
    df = pd.read_excel('first_15000_rows_imdb.xlsx', usecols=[0, 1])  # Read the first two columns of the Excel sheet
    narrativity_scores = []
    run_again = True  # Flag to indicate if it can run again
    start_index = 0  # Start index for resuming the process

    while run_again:
        try:
            with ThreadPoolExecutor() as executor:
                future_to_review = {executor.submit(process_review, review): review for review in df.iloc[start_index:, :].itertuples(index=False)}

                for future in concurrent.futures.as_completed(future_to_review):
                    result = future.result()
                    narrativity_scores.append(result)
                    print(f"Processed review {len(narrativity_scores)}/{len(df)}")
                    time.sleep(2)  # Pause for 2 seconds between API calls

            dp = pd.DataFrame(narrativity_scores, columns=['Review ID', 'Review', 'Narrativity Score'])
            dp.to_csv('/Users/muskan/Documents/file_final.csv', index=False)

            run_again = False  # Set the flag to False if the loop completes successfully
        except Exception:
            print("Pausing for 2 minutes...")
            time.sleep(120)  # Pause for 2 minutes after a crash

            # Get the index of the last processed review
            start_index = len(narrativity_scores)

    print("Finished processing.")

func(narrativity=True)


Processed review 1/14425
Processed review 2/14425
Processed review 3/14425
Processed review 4/14425
Processed review 5/14425
Processed review 6/14425
Processed review 7/14425
Processed review 8/14425
Processed review 9/14425
Processed review 10/14425
Processed review 11/14425
Processed review 12/14425
Processed review 13/14425
Processed review 14/14425
Processed review 15/14425
Processed review 16/14425
Processed review 17/14425
Processed review 18/14425
Processed review 19/14425
Processed review 20/14425
Processed review 21/14425
Processed review 22/14425
Processed review 23/14425
Processed review 24/14425
Processed review 25/14425
Processed review 26/14425
Processed review 27/14425
Processed review 28/14425
Processed review 29/14425
Processed review 30/14425
Processed review 31/14425
Processed review 32/14425
Processed review 33/14425
Processed review 34/14425
Processed review 35/14425
Processed review 36/14425
Processed review 37/14425
Processed review 38/14425
Processed review 39/1